In [1]:
import numpy as np 
import pandas as pd
from sklearn.svm import SVR
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import RFECV
from sklearn.svm import LinearSVR
%matplotlib inline

In [2]:
train = pd.read_csv("mertrain.csv")
test = pd.read_csv("mertest.csv")

# process columns, apply LabelEncoder to categorical features
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder() 
        lbl.fit(list(train[c].values) + list(test[c].values)) 
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

# shape        
print('Shape train: {}\nShape test: {}'.format(train.shape, test.shape))

Shape train: (4209, 378)
Shape test: (4209, 377)


In [3]:
y = train['y'].values
y_mean = np.mean(y)
id_test = test['ID'].values
X = train.drop(['y'], axis=1).values
X_test= test.values

In [4]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=5, shuffle=True, random_state= 0)
kf.get_n_splits(X)

5

In [19]:
scl = ('scl', MaxAbsScaler())
rdm = ('rdm', RFECV(cv=kf, step=5, estimator=SVR(C=100, kernel='linear'), scoring='r2', n_jobs=2))
pipe=Pipeline([scl, rdm])

In [20]:
pipe.fit(X,y)

Pipeline(memory=None,
     steps=[('scl', MaxAbsScaler(copy=True)), ('rdm', RFECV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
   estimator=SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
   n_jobs=2, scoring='r2', step=5, verbose=0))])

In [21]:
y_pred=pipe.predict(X_test)

In [22]:
sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = y_pred
sub.to_csv('svr-maxabs-vif-c10.csv', index=False)
